In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
import matplotlib.pyplot as plt
import os

In [2]:
class SpiralDataset(torch.utils.data.Dataset):
    def __init__(self, dir, train=True):
        # code for generating this dataset is in spiral_dataset_generator.ipynb
        super().__init__()

        dataset_file = os.path.join(
            dir, 'spiral', 'train_dataset.txt' if train else 'test_dataset.txt')

        self.data, self.targets = [], []

        with open(dataset_file, 'r') as f:
            for line in f:
                data_x, data_y, target = line.split(' ')
                data_x, data_y = float(data_x), float(data_y)
                self.data.append([data_x, data_y, math.sin(data_x), math.sin(data_y)])
                self.targets.append(float(target))

        self.data, self.targets = torch.tensor(
            self.data), torch.tensor(self.targets)

    def __getitem__(self, index):
        # print(type(self.targets[index]))
        return self.data[index], self.targets[index]

    def __len__(self):
        return len(self.targets)


In [3]:
dir="datasets"
trainset = SpiralDataset(dir, train=True)
testset = SpiralDataset(dir, train=False)

In [4]:
train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=1, pin_memory=True)
test_loader = torch.utils.data.DataLoader(
    testset, batch_size=64, shuffle=False, num_workers=1, pin_memory=True)


In [5]:
class SpiralModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(4, 12, bias=False), torch.nn.Tanh())
        self.layer2 = nn.Sequential(
            nn.Linear(12, 24, bias=False), torch.nn.Tanh())
        self.layer3 = nn.Sequential(
            nn.Linear(24, 24, bias=False), torch.nn.Tanh())
        self.layer4 = nn.Sequential(
            nn.Linear(24, 32, bias=False), torch.nn.Tanh())
        self.layer5 = nn.Sequential(
            nn.Linear(32, 2, bias=False), torch.nn.Tanh())

    def forward(self, x):
        out = x.float()
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        return out


In [6]:
def train(model, train_loader, batch_size, optimizer, epoch):
    model.train()
    loss_per_batch = []

    acc_train_per_batch = []
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()

        output = model(data)
        output = F.log_softmax(output, dim=1)

        loss = F.nll_loss(output, target.long())

        loss.backward()
        optimizer.step()
        loss_per_batch.append(loss.item())

        # save accuracy:
        # get the index of the max log-probability
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(target.view_as(pred)).sum().item()
        acc_train_per_batch.append(
            100. * correct / ((batch_idx+1)*batch_size))

        # if batch_idx % 1000 == 0:
        #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {:.0f}%, Learning rate: {:.6f}'.format(
        #         epoch, batch_idx * len(data), len(train_loader.dataset),
        #         100. * batch_idx / len(train_loader), loss.item(),
        #         100. * correct / ((batch_idx + 1) * batch_size),
        #         optimizer.param_groups[0]['lr']))

    loss_per_epoch = [np.average(loss_per_batch)]
    acc_train_per_epoch = [np.average(acc_train_per_batch)]
    return (loss_per_epoch, acc_train_per_epoch)


In [7]:
model = SpiralModel()

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.001)

In [9]:
for epoch in range(100):
    loss_per_epoch, acc_train_per_epoch_i =  train(
        model, train_loader, 64, optimizer, epoch)
    if epoch % 10 == 0:
        print(acc_train_per_epoch_i)


/home/masonwang/anaconda3/envs/lnoise-analysis/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


[48.2444659019366]
[63.02338747407107]
[84.1808035985282]
[91.67852573858434]
[88.27870005970593]
[92.41918427709443]
[91.85762208711428]
[91.86411190805723]
[88.7281281543977]
[94.95233824335386]


In [10]:
import visualizer

In [11]:
demo_loader = torch.utils.data.DataLoader(
    trainset, batch_size=len(trainset), num_workers=1, pin_memory=True)
x_train, y_train = next(iter(train_loader))


In [12]:
visualizer.plot_spiral_dataset(
    x_train, y_train, f'Spiral Dataset (noise: 0%)')
print(list(y_train).count(0) / len(y_train))


NameError: name 'NOISE_LEVEL' is not defined